# RouterChain

In [1]:
from langchain.prompts import PromptTemplate
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    model_name="gpt-4o",
    temperature=0.8,
    max_tokens=512
) 



# 目标链

In [2]:
# 构建两个场景的模板
flower_care_template = """你是一个经验丰富的园丁，擅长解答关于养花育花的问题。
                        下面是需要你来回答的问题:
                        {input}"""

flower_deco_template = """你是一位网红插花大师，擅长解答关于鲜花装饰的问题。
                        下面是需要你来回答的问题:
                        {input}"""

# 构建提示信息
prompt_infos = [
    {
        "key": "flower_care",
        "description": "适合回答关于鲜花护理的问题",
        "template": flower_care_template,
    },
    {
        "key": "flower_decoration",
        "description": "适合回答关于鲜花装饰的问题",
        "template": flower_deco_template,
    }]

chain_map = {}

for info in prompt_infos:
    prompt = PromptTemplate.from_template(info['template'])
    chain_map[info["key"]] = prompt | llm


# 路由链

In [3]:
from langchain.chains.router.llm_router import RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE as RouterTemplate

destinations = [f"{p['key']}: {p['description']}" for p in prompt_infos]
router_template = RouterTemplate.format(destinations="\n".join(destinations))
# 
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser()
)

router_chain = router_prompt | llm

# 默认链

In [4]:
# from langchain.chains import ConversationChain
# from langchain.chains.conversation.base import ConversationChain
from langchain_core.output_parsers import StrOutputParser

default_chain = llm | {"text": StrOutputParser()}

# 多提示链

In [13]:
# 构建目标链
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

chain_map = {}

for info in prompt_infos:
    prompt = PromptTemplate(
        template=info['template'],
        input_variables=["input"]
    )
    
    chain = LLMChain(
        llm=llm,
        prompt=prompt,
        verbose=True
    )
    # chain = prompt | llm
    chain_map[info["key"]] = chain

# 构建路由链
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE as RounterTemplate

destinations = [f"{p['key']}: {p['description']}" for p in prompt_infos]
router_template = RounterTemplate.format(destinations="\n".join(destinations))

router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(
    llm,
    router_prompt,
    verbose=True
)

# 构建默认链
from langchain.chains import ConversationChain

default_chain = ConversationChain(
    llm=llm,
    output_key="text",
    verbose=True
)

# 构建多提示链
from langchain.chains.router import MultiPromptChain

chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=chain_map,
    default_chain=default_chain,
    verbose=True
)

ValidationError: 2 validation errors for MultiPromptChain
destination_chains -> flower_care
  Can't instantiate abstract class Chain without an implementation for abstract methods '_call', 'input_keys', 'output_keys' (type=type_error)
destination_chains -> flower_decoration
  Can't instantiate abstract class Chain without an implementation for abstract methods '_call', 'input_keys', 'output_keys' (type=type_error)

In [12]:
# 测试1
print(chain.invoke("如何为玫瑰浇水？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
flower_care: {'input': '如何为玫瑰浇水？'}

> Entering new LLMChain chain...
Prompt after formatting:
你是一个经验丰富的园丁，擅长解答关于养花育花的问题。
                        下面是需要你来回答的问题:
                        如何为玫瑰浇水？

> Finished chain.

> Finished chain.
{'input': '如何为玫瑰浇水？', 'text': '当然，我很高兴能够帮助你！为玫瑰浇水时需要注意以下几点：\n\n1. **浇水频率**:\n   - **春夏季节**：这些是玫瑰的生长旺季，需要充足的水分。通常，每周浇水1-2次，但在非常炎热干燥的天气下，可能需要更频繁地浇水。\n   - **秋冬季节**：随着温度降低，玫瑰的生长速度减缓，对水的需求也减少。每周浇水一次或更少即可。\n\n2. **浇水量**:\n   - 每次浇水应确保土壤彻底湿润，但不要出现积水。一般来说，每次浇水的水量应足以渗透至少15厘米深的土壤层。\n   - 确保根部周围的土壤完全湿润，但不要让土壤变得过于泥泞。\n\n3. **浇水时间**:\n   - 最佳浇水时间是早晨。这可以使玫瑰在白天吸收水分，并且早晨的低温可以减少水分蒸发。\n   - 避免在中午或下午高温时段浇水，以免水分迅速蒸发。\n\n4. **浇水方法**:\n   - 尽量避免将水直接洒在叶子和花朵上，因为这可能会导致霉菌和病害。最好直接浇在根部周围的土壤上。\n   - 使用软喷头或漫灌的方式浇水，确保水流均匀地渗透到土壤中。\n\n5. **土壤排水**:\n   - 玫瑰喜欢排水良好的土壤，确保你的花盆或花坛有良好的排水系统，防止积水引起根部腐烂。\n\n6. **观察植株状态**:\n   - 经常观察玫瑰的叶子和花朵。如果叶子开始卷曲、变黄或者掉落，可能是缺水的信号。如果叶子变黄且潮湿、根部有腐烂味道，则可能是浇水过多。\n\n通过这些

In [ ]:
# 测试2              
print(chain.run("如何为婚礼场地装饰花朵？"))

In [ ]:
# 测试3         
print(chain.run("如何区分阿豆和罗豆？"))

# 说明

由于 LLMChain deprecated，上面的demo，需要参考该链接改写：https://python.langchain.com/v0.2/docs/how_to/routing/